# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [2]:
# Import the WeatherPy_database.csv file. 
filename = '../Weather_Database/Weatherpy_Database.csv'
weather_data = pd.read_csv(filename)
city_data_df = pd.DataFrame(weather_data)

# Display sample data
city_data_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Nanortalik,GL,60.1432,-45.2371,20.03,64,99,6.20,overcast clouds
1,1,Bluff,NZ,-46.6000,168.3333,55.98,78,100,7.87,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,68.11,32,0,21.85,clear sky
3,3,Lebu,CL,-37.6167,-73.6500,64.51,65,4,9.28,clear sky
4,4,Nemuro,JP,43.3236,145.5750,23.29,61,96,9.24,overcast clouds
5,5,Hilo,US,19.7297,-155.0900,71.26,93,100,8.05,overcast clouds
6,6,Ambon,ID,-3.6954,128.1814,78.24,15,40,2.30,scattered clouds
7,7,Rikitea,PF,-23.1203,-134.9692,77.56,73,98,8.86,overcast clouds
8,8,Olinda,BR,-8.0089,-34.8553,83.93,61,75,11.50,broken clouds
9,9,Cape Town,ZA,-33.9258,18.4232,75.90,67,0,10.36,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [3]:
# Prompt the user to enter minimum and maximum temperature criteria.
max_temp = int(input('Enter maximum destination temperature (˚F): '))
min_temp = int(input('Enter minimum destination temperature (˚F): '))


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<=max_temp) & (city_data_df['Max Temp']>=min_temp)]

# Display sample data
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
6,6,Ambon,ID,-3.6954,128.1814,78.24,15,40,2.30,scattered clouds
7,7,Rikitea,PF,-23.1203,-134.9692,77.56,73,98,8.86,overcast clouds
8,8,Olinda,BR,-8.0089,-34.8553,83.93,61,75,11.50,broken clouds
9,9,Cape Town,ZA,-33.9258,18.4232,75.90,67,0,10.36,clear sky
10,10,Isangel,VU,-19.5500,169.2667,76.46,81,100,4.59,overcast clouds
11,11,Mamallapuram,IN,12.6264,80.1722,77.05,69,1,8.59,clear sky
18,18,Port Alfred,ZA,-33.5906,26.8910,75.18,96,61,3.00,broken clouds
26,26,Hermanus,ZA,-34.4187,19.2345,78.80,73,12,17.20,few clouds
28,28,Salalah,OM,17.0151,54.0924,77.09,61,0,1.14,clear sky
29,29,Kalmunai,LK,7.4167,81.8167,76.03,78,100,9.22,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [5]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
print(f'Length before cleaning: {len(preferred_cities_df)}')
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
print(f'Length after cleaning: {len(clean_travel_cities)}')
clean_travel_cities.head(10)

Length before cleaning: 212
Length after cleaning: 211


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
6,6,Ambon,ID,-3.6954,128.1814,78.24,15,40,2.30,scattered clouds
7,7,Rikitea,PF,-23.1203,-134.9692,77.56,73,98,8.86,overcast clouds
8,8,Olinda,BR,-8.0089,-34.8553,83.93,61,75,11.50,broken clouds
9,9,Cape Town,ZA,-33.9258,18.4232,75.90,67,0,10.36,clear sky
10,10,Isangel,VU,-19.5500,169.2667,76.46,81,100,4.59,overcast clouds
11,11,Mamallapuram,IN,12.6264,80.1722,77.05,69,1,8.59,clear sky
18,18,Port Alfred,ZA,-33.5906,26.8910,75.18,96,61,3.00,broken clouds
26,26,Hermanus,ZA,-34.4187,19.2345,78.80,73,12,17.20,few clouds
28,28,Salalah,OM,17.0151,54.0924,77.09,61,0,1.14,clear sky
29,29,Kalmunai,LK,7.4167,81.8167,76.03,78,100,9.22,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[['City','Country','Max Temp','Description','Lat','Lng']].copy()
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng
6,Ambon,ID,78.24,scattered clouds,-3.6954,128.1814
7,Rikitea,PF,77.56,overcast clouds,-23.1203,-134.9692
8,Olinda,BR,83.93,broken clouds,-8.0089,-34.8553
9,Cape Town,ZA,75.90,clear sky,-33.9258,18.4232
10,Isangel,VU,76.46,overcast clouds,-19.5500,169.2667
11,Mamallapuram,IN,77.05,clear sky,12.6264,80.1722
18,Port Alfred,ZA,75.18,broken clouds,-33.5906,26.8910
26,Hermanus,ZA,78.80,few clouds,-34.4187,19.2345
28,Salalah,OM,77.09,clear sky,17.0151,54.0924
29,Kalmunai,LK,76.03,overcast clouds,7.4167,81.8167


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [7]:
# Create a new column "Hotel Name".
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Ambon,ID,78.24,scattered clouds,-3.6954,128.1814,
7,Rikitea,PF,77.56,overcast clouds,-23.1203,-134.9692,
8,Olinda,BR,83.93,broken clouds,-8.0089,-34.8553,
9,Cape Town,ZA,75.90,clear sky,-33.9258,18.4232,
10,Isangel,VU,76.46,overcast clouds,-19.5500,169.2667,
11,Mamallapuram,IN,77.05,clear sky,12.6264,80.1722,
18,Port Alfred,ZA,75.18,broken clouds,-33.5906,26.8910,
26,Hermanus,ZA,78.80,few clouds,-34.4187,19.2345,
28,Salalah,OM,77.09,clear sky,17.0151,54.0924,
29,Kalmunai,LK,76.03,overcast clouds,7.4167,81.8167,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [8]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

In [9]:
# unecessary, but I like the function format for repetitive tasks
def call_geo(params):
    url = 'https://api.geoapify.com/v2/places'
    response = requests.get(url, params).json()
    return response

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [11]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = call_geo(params)['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
print('--------------------------\n Hotel Search Complete')
# Display sample data
hotel_df

Starting hotel search
Ambon - nearest hotel: Hotel Hero
Rikitea - nearest hotel: Chez Bianca & Benoit
Olinda - nearest hotel: Fidji Motel
Cape Town - nearest hotel: Townhouse Hotel
Isangel - nearest hotel: Tanna Lodge
Mamallapuram - nearest hotel: Radisson Temple Bay
Port Alfred - nearest hotel: No hotel found
Hermanus - nearest hotel: Aloe guest house
Salalah - nearest hotel: Muscat International Hotel
Kalmunai - nearest hotel: Saji new food
Atar - nearest hotel: فندق سكليل
Mumford - nearest hotel: No hotel found
Georgetown - nearest hotel: Page 63 hostel
Meulaboh - nearest hotel: Hotel Meuligou
Atuona - nearest hotel: Pearl Resort
Broken Hill - nearest hotel: Ibis Styles
Butaritari - nearest hotel: No hotel found
Mar Del Plata - nearest hotel: Nuevo Ostende
Bellavista - nearest hotel: No hotel found
Katherine - nearest hotel: No hotel found
Baracoa - nearest hotel: Casa
Vaini - nearest hotel: Keleti International Resort
Axim - nearest hotel: Axim Beach Hotel
Hithadhoo - nearest hotel

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Ambon,ID,78.24,scattered clouds,-3.6954,128.1814,Hotel Hero
7,Rikitea,PF,77.56,overcast clouds,-23.1203,-134.9692,Chez Bianca & Benoit
8,Olinda,BR,83.93,broken clouds,-8.0089,-34.8553,Fidji Motel
9,Cape Town,ZA,75.90,clear sky,-33.9258,18.4232,Townhouse Hotel
10,Isangel,VU,76.46,overcast clouds,-19.5500,169.2667,Tanna Lodge
...,...,...,...,...,...,...,...
684,Saposoa,PE,86.81,overcast clouds,-6.9340,-76.7716,No hotel found
686,Mapiripan,CO,88.54,overcast clouds,2.8892,-72.1256,No hotel found
687,Kulhudhuffushi,MV,79.97,few clouds,6.6221,73.0700,No hotel found
688,Suba,PH,79.86,overcast clouds,10.8438,121.0102,Jade Felimar Value Inn


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [12]:
# Drop the rows where there is no Hotel Name.
print(f'Cities searced: {len(hotel_df)}')
clean_hotel_df = hotel_df.drop(hotel_df[(hotel_df['Hotel Name'] == 'No hotel found')].index)

# Display sample data
print(f'Hotels found: {len(clean_hotel_df)}')
clean_hotel_df

Cities searced: 211
Hotels found: 139


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Ambon,ID,78.24,scattered clouds,-3.6954,128.1814,Hotel Hero
7,Rikitea,PF,77.56,overcast clouds,-23.1203,-134.9692,Chez Bianca & Benoit
8,Olinda,BR,83.93,broken clouds,-8.0089,-34.8553,Fidji Motel
9,Cape Town,ZA,75.90,clear sky,-33.9258,18.4232,Townhouse Hotel
10,Isangel,VU,76.46,overcast clouds,-19.5500,169.2667,Tanna Lodge
...,...,...,...,...,...,...,...
675,Tres Arroyos,AR,86.11,moderate rain,-38.3739,-60.2798,Andrea Hotel
676,Mayumba,GA,79.03,overcast clouds,-3.4320,10.6554,Résidence Hôtelière Fabso
680,Rocha,UY,88.03,clear sky,-34.4833,-54.3333,Hotel Municipal
688,Suba,PH,79.86,overcast clouds,10.8438,121.0102,Jade Felimar Value Inn


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [13]:
# Create the output File (CSV)
filename = 'Weatherpy_vacation.csv'

# Export the City_Data into a csv
clean_hotel_df.to_csv(filename)

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [35]:
# Configure the map plot
city_plot = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ['City',"Hotel Name", "Country", "Description"],
    alpha = .7
)

# Display the map
city_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Description)